In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('./high_diamond_ranked_10min.csv')
df.head(3)
original = df

In [3]:
df = df[df['blueWardsPlaced'] <= 50]  # drop the edge cases, wards placed > 50
df = df[df['redWardsPlaced'] <= 50]

df = df.drop(['blueEliteMonsters', 'blueGoldPerMin', 'redFirstBlood', 'redKills',
              'redDeaths', 'redEliteMonsters', 'redGoldDiff', 'redGoldPerMin'], axis = 1)
test_id = df.gameId
df = df.drop(df.columns[0], axis = 1)    # drop the 'gameId' column

df.shape

(8780, 31)

In [4]:
X = df.drop('blueWins', axis = 1)
y = df.blueWins

X = (X - X.mean())/X.std()

In [5]:
pca = PCA(n_components=20)
X_pca = pca.fit_transform(X)
rf = RandomForestClassifier(n_estimators=121, max_depth=2, oob_score=True)
rf.fit(X_pca, y)

RandomForestClassifier(max_depth=2, n_estimators=121, oob_score=True)

In [6]:
rf.oob_score_

0.7310933940774488

In [7]:
yh_test = rf.predict(X_pca)

In [8]:
demo = pd.DataFrame()
demo['id'] = test_id
demo['Predict_Type'] = yh_test.tolist()
demo['Origin_Type'] = y.tolist()
demo['Correct'] = (demo.Predict_Type==demo.Origin_Type)

In [9]:
demo

,id,Predict_Type,Origin_Type,Correct
0,4519157822,1,0,False
1,4523371949,0,0,True
2,4521474530,0,0,True
3,4524384067,0,0,True
5,4475365709,1,1,True
...,...,...,...,...
9871,4527715781,0,0,True
9873,4527878058,1,1,True
9874,4527873286,1,1,True
9876,4527713716,0,0,True


In [10]:
op = pd.DataFrame()
op = pd.concat([op,original.loc[original['gameId']==demo['id'].iloc[0]]])
op = pd.concat([op,original.loc[original['gameId']==demo['id'].iloc[1]]])
op = pd.concat([op,original.loc[original['gameId']==demo['id'].iloc[2]]])
op = pd.concat([op,original.loc[original['gameId']==demo['id'].iloc[3]]])
op = pd.concat([op,original.loc[original['gameId']==demo['id'].iloc[4]]])
op

,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
5,4475365709,1,18,0,0,5,3,6,1,1,...,0,15201,7.0,18060,221,59,-698,-101,22.1,1520.1
